In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import scipy.stats as sps
import sklearn
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D
import time

In [2]:
#Parameters
S0 = 100
r = 0.05
sigma = 0.4
rho = 0.75 # Correlation of brownian motions
T = 1
G = 100
range_K = np.arange(99, 102)

nb_sample = 100000 

# Default value ok K
fixed_K = 100
ind_K = list(range_K).index(fixed_K)

### Payoff and call price

In [3]:
def f(Z, K):
    ## Function computing the payoff
    S = S0*np.exp(sigma*np.sqrt(T)*Z - 0.5*T*sigma**2 + r*T)
    return np.maximum( 0, G*((np.mean(S,axis=1)>=K).astype(int)) )

def Call(K):
    #Calculates the European call option for a continuous time
    
    k = K*np.exp(-r*T)
    v = T*sigma**2
    d1 = ( np.log(S0/k)/np.sqrt(v) ) + np.sqrt(v)/2     #d+
    d2 = ( np.log(S0/k)/np.sqrt(v) ) - np.sqrt(v)/2     #d-
    
    return S0*sps.norm.cdf(d1) - k*sps.norm.cdf(d2)

### Model training (weigths, biases and analytical expectation)

In [4]:
def NN_model(Z_train, K, N, n, A):
    # defining our model
    # Z_train is a training vector
    # K is a fixed strike
    model = keras.Sequential()
    model.add( layers.Dense(n, input_dim = N, activation = 'relu') )
    model.add( layers.Dense(1))

    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    # Fitting the model
    Y_train = f(Z_train, K)
    model.fit(Z_train, Y_train)

    # Getting the weights:
    W1, b1 = model.layers[0].get_weights()
    W2, b2 = model.layers[1].get_weights()
    
    # Calculation the "esperance"
    mu = b1
    sig = np.sqrt(np.sum((A.T@W1)**2, axis=0))
    esp1 = (sig/(np.sqrt(2*np.pi)))*np.exp(-0.5*mu**2/sig**2) + mu*(1 - sps.norm.cdf(-mu/sig))
    esp2 = np.dot(W2[:, 0], esp1) + b2[0]
    
    return (W1, b1), (W2, b2), esp2

### Computing the contole variate function

In [5]:
def NN_h(Z, K, W1, b1, W2, b2):
    #value = (np.maximum(0, Z@W1 + b1)@W2 + b2)[0]
    value = (np.maximum(0, Z@W1 + b1)@W2 + b2).flatten()
    return value

def NN_f_ctrl(Z, K, W1, b1, W2, b2, esp2):
    return f(Z, K) - NN_h(Z, K, W1, b1, W2, b2) + esp2

### DIMENSION 10

In [6]:
N = 10
n = 500

# Sampling correlated bownian motions
Cov = np.eye(N)
Cov[Cov != 1] = rho
A = np.linalg.cholesky(Cov)
Z = np.random.multivariate_normal(np.zeros(N), Cov, nb_sample)

### Naive Monte Carlo Method VS Controle variate

In [7]:
# Option price with the naive Monte Carlo Method
ti_MC = time.time()

payoffs_MC = f(Z, fixed_K)
prices_MC = np.mean(payoffs_MC)*np.exp(-r*T)

tf_MC = time.time()

std_MC = np.std(payoffs_MC)*np.exp(-r*T)
IC_MC = prices_MC + 1.96 / np.sqrt(nb_sample) * np.array([-std_MC,std_MC])

In [8]:
# Option Price with Controle Variate Method
ti_train = time.time()
(W1, b1), (W2, b2), esp2 = NN_model(Z, fixed_K, N, n, A)
tf_train = time.time()

3125/3125 [==============================] - 11s 3ms/step - loss: 476.0803


In [9]:
ti_compute = time.time()

payoffs_ctrl = NN_f_ctrl(Z, fixed_K, W1, b1, W2, b2, esp2)
prices_ctrl = np.mean(payoffs_ctrl)*np.exp(-r*T)

tf_compute = time.time()

std_ctrl = np.std(payoffs_ctrl)*np.exp(-r*T)
IC_ctrl = prices_ctrl + 1.96 / np.sqrt(nb_sample) * np.array([-std_ctrl,std_ctrl])

In [10]:
# Connfidence intervalls, Ratio of variance, cost and speed up
std_ctrl = np.std(payoffs_ctrl)*np.exp(-r*T)
IC_ctrl = prices_ctrl + 1.96 / np.sqrt(nb_sample) * np.array([-std_ctrl,std_ctrl])
var_ratio = (std_MC/std_ctrl)**2
cost = ((tf_train - ti_train) + (tf_compute - ti_compute))/(tf_MC - ti_MC)
speed_up = var_ratio/cost

In [11]:
print("Comparison between the Naive Monte Carlo Method and the Controle variate one")
print("==============================================================================\n\n")
print(f"Naive Monte Carlo price = {prices_MC}, IC = {IC_MC}\n")
print(f"Controle variate price = {prices_ctrl}, IC = {IC_ctrl}\n")
print(f"var_ratio = {var_ratio}\n")
print(f"Cost = {cost}\n")
print(f"Speed up = {speed_up}\n")

Comparison between the Naive Monte Carlo Method and the Controle variate one


Naive Monte Carlo price = 46.2202377364897, IC = [45.92556592 46.51490955]

Controle variate price = 46.28327970168827, IC = [46.22375193 46.34280747]

var_ratio = 24.504055531795245

Cost = 192.6130575762806

Speed up = 0.12721907766866167



#### Naive Monte Carlo method

payoffs_MC = np.zeros( (len(range_K), nb_sample) )

for i,K in enumerate(range_K):
    payoffs_MC[i,:] = f(Z, K)[:]
    
prices_MC = np.mean(payoffs_MC, axis =1)*np.exp(-r*T)
std_MC = np.std(payoffs_MC , axis = 1)*np.exp(-r*T)
IC_MC = prices_MC + 1.96 / np.sqrt(payoffs_MC.shape[1]) * np.array([-std_MC,std_MC])

#### Calculating the price option with NN controle variate in dim=10

payoffs_ctrl = np.zeros( (len(range_K), nb_sample) )

for i,K in enumerate(range_K):
    (W1, b1), (W2, b2), esp2 = NN_model(Z, K, N, n, A)
    payoffs_ctrl[i,:] = NN_f_ctrl(Z, K, W1, b1, W2, b2, esp2)[:]

prices_ctrl = np.mean(payoffs_ctrl, axis =1)*np.exp(-r*T)
std_ctrl = np.std(payoffs_ctrl , axis = 1)*np.exp(-r*T)
IC_ctrl = prices_ctrl + 1.96 / np.sqrt(payoffs_ctrl.shape[1]) * np.array([-std_ctrl,std_ctrl])

#true_prices = np.array([Call(K) for K in range_K])

plt.figure(figsize=(8, 5))
plt.plot(range_K, prices_ctrl, 'r', label=f'Monte Carlo Prices with NN')
plt.plot(range_K, prices_MC, 'b', label='Naive Monte Carlo prices')
plt.fill_between(range_K, IC_ctrl[0], IC_ctrl[1], alpha=0.1, label='Confidence interval for NN Monte carlo', color='r')
plt.fill_between(range_K, IC_MC[0], IC_MC[1], alpha=0.1, label='Confidence interval for naive Monte Carlo', color='b')
plt.xlabel('Strike Prices (K)')
plt.ylabel('Call Prices')
plt.title(f'Monte Carlo estimation of the call price using NN with {nb_sample} samples of N={N} correlated assets')
plt.legend(loc='best')
plt.grid()
plt.show()